# **load evaluative data**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import json
import math

In [4]:
# %cd '/content/drive/MyDrive/巨量行銷/kprototypes'
# !unzip drive-download-20220514T063310Z-001.zip

Archive:  drive-download-20220514T063310Z-001.zip
  inflating: outputs/set3_k10.json   
  inflating: outputs/set4_k20.json   
  inflating: outputs/set2_k20.json   
  inflating: outputs/set4_k30.json   
  inflating: outputs/set1_k20.json   
  inflating: outputs/set1_k10.json   
  inflating: outputs/set3_k20.json   
  inflating: outputs/set2_k10.json   
  inflating: outputs/set4_k10.json   
  inflating: outputs/set1_k30.json   
  inflating: outputs/set2_k30.json   
  inflating: outputs/set3_k30.json   
  inflating: results/set1_k20.csv    
  inflating: results/set1_k10.csv    
  inflating: results/set2_k20.csv    
  inflating: results/set3_k10.csv    
  inflating: results/set1_k30.csv    
  inflating: results/set3_k30.csv    
  inflating: results/set2_k30.csv    
  inflating: results/set3_k20.csv    
  inflating: results/set4_k20.csv    
  inflating: results/set2_k10.csv    
  inflating: results/set4_k10.csv    
  inflating: results/set4_k30.csv    


# **set 1**

# **calculate evaluative scores(based on Calinski-Harabasz Score)**

**load data**

In [3]:
set1_k10_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set1_k10.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
set1_k20_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set1_k20.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
set1_k30_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set1_k30.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})

In [4]:
with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set1_k10.json") as f:
  set1_k10_json = json.load(f)

with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set1_k20.json") as f:
  set1_k20_json = json.load(f)

with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set1_k30.json") as f:
  set1_k30_json = json.load(f)

**count per cluster**

In [13]:
count_per_cluster_df = set1_k10_df['cluster_labels'].value_counts().rename_axis('cluster_num').reset_index(name='counts')
count_per_cluster_df = count_per_cluster_df.sort_values(by=['cluster_num'])
count_per_cluster = count_per_cluster_df['counts'].values.tolist()
print(count_per_cluster)
print(len(count_per_cluster))

[6916, 6, 566, 8692, 22452, 4137, 6188, 5500, 14, 709, 7449, 15554, 2, 10656, 294, 36847, 27705, 16395, 24261, 291, 32019, 21028, 476, 21643, 3744, 11506, 87, 20, 5111, 18623]
30


**find the centroid of the dataset**

In [14]:
temp = []

categorical_feature = set1_k10_df[['Gender_Code', 'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code']]
numerical_feature = set1_k10_df.drop(columns=['Gender_Code', 'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code', 'cluster_labels'])

temp.append(set1_k10_df[list(numerical_feature.columns)].mean().values.tolist())
temp.append(categorical_feature.mode().values.tolist())
#print(temp)

dataset_centroids = []
for i in range(len(temp[0])):
  dataset_centroids.append(temp[0][i])

for i in range(len(temp[1][0])):
  dataset_centroids.append(temp[1][0][i])

print(dataset_centroids)
print(len(dataset_centroids))

[-1.9670794114423322e-16, -7.527986994143146e-17, 1.718306190164684e-16, -1.758796545861717e-15, -1.5461310649084406e-15, -1.6862261357275505e-15, -1.8584981140519205e-16, -4.1207910797100547e-16, 'F', '300', 'P02', '23:00-23:59', 'unknown', '00', 'unknown']
15


**calculate the distance between clusters**

In [15]:
#將群心的數值型資料從str轉為float
for i in range(len(set1_k10_json['cluster_centroids'])):
  for j in range(0,8):
    set1_k10_json['cluster_centroids'][i][j] = float(set1_k10_json['cluster_centroids'][i][j])

In [16]:
## 計算群間總距離

#for numeric data(Euclidean distance) 
#計算資料中心點與所有群心的距離平方和加總
def SquareDsit(x, y):
  return sum([(x-y)**2 for (x,y) in zip(x,y)])

numeric_dist_between_cluters = 0
for i in range(len(set1_k10_json['cluster_centroids'])):
  distance = count_per_cluster[i]*(SquareDsit(set1_k10_json['cluster_centroids'][i][:8], dataset_centroids[:8]))
  numeric_dist_between_cluters += distance



#for categorical data
temp = 0
categorical_dist_between_cluters = 0
for i in range(len(set1_k10_json['cluster_centroids'])):
  for j in range(len(set1_k10_json['cluster_centroids'][i][8:])):
    if (set1_k10_json['cluster_centroids'][i][8+j] == dataset_centroids[8+j]):
      temp += 0
    elif (set1_k10_json['cluster_centroids'][i][8+j] != dataset_centroids[8+j]):
      temp += 1    
  distance = count_per_cluster[i]*temp
  temp = 0
  categorical_dist_between_cluters += distance



#mix numeric and categorical
categorical_weight = set1_k10_json['categorical_weight'] 
dist_between_cluters = numeric_dist_between_cluters + (categorical_weight * categorical_dist_between_cluters)

**calculate evaluative scores**

In [17]:
dist_within_cluster = set1_k10_json['dist_within_cluster'] 
N = len(set1_k10_df)
k = 10

In [18]:
#evaluate score(based on Calinski-Harabasz Score) -> (distance between clusters/distance within cluster)*(N-k/k-1)
evaluate_score = (dist_between_cluters/(dist_within_cluster))*((N-k)/(k-1))
evaluate_score

23664.8529175796

# **set 2**

# **calculate evaluative scores(based on Calinski-Harabasz Score)**

**load data**

In [19]:
set2_k10_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set2_k10.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
set2_k20_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set2_k20.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
set2_k30_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set2_k30.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})

In [20]:
with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set2_k10.json") as f:
  set2_k10_json = json.load(f)

with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set2_k20.json") as f:
  set2_k20_json = json.load(f)

with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set2_k30.json") as f:
  set2_k30_json = json.load(f)

**count per cluster**

In [33]:
count_per_cluster_df = set2_k30_df['cluster_labels'].value_counts().rename_axis('cluster_num').reset_index(name='counts')
count_per_cluster_df = count_per_cluster_df.sort_values(by=['cluster_num'])
count_per_cluster = count_per_cluster_df['counts'].values.tolist()
print(count_per_cluster)
print(len(count_per_cluster))

[23876, 914, 6527, 15, 4790, 6010, 1699, 13167, 9151, 910, 396, 6298, 9722, 9506, 1993, 18021, 11047, 556, 435, 14649, 15621, 35791, 74, 22119, 6634, 42373, 27089, 6, 19495, 7]
30


**find the centroid of the dataset**

In [34]:
temp = []

categorical_feature = set2_k30_df[['Gender_Code', 'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code']]
numerical_feature = set2_k30_df.drop(columns=['Gender_Code', 'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code', 'cluster_labels'])

temp.append(set2_k30_df[list(numerical_feature.columns)].mean().values.tolist())
temp.append(categorical_feature.mode().values.tolist())
#print(temp)

dataset_centroids = []
for i in range(len(temp[0])):
  dataset_centroids.append(temp[0][i])

for i in range(len(temp[1][0])):
  dataset_centroids.append(temp[1][0][i])

print(dataset_centroids)
print(len(dataset_centroids))

[-1.7139122529907155e-16, -7.527986994143146e-17, -1.4407941936690026e-15, 1.8491233143792758e-16, 6.613684236771283e-15, 3.2822980256241275e-16, -2.974190478496039e-16, 3.1704655853791885e-15, -3.135276708072015e-15, -4.856565743603306e-16, 2.46474482276692e-15, 9.185743786567105e-16, -7.636462969578343e-15, 1.718306190164684e-16, -1.758796545861717e-15, -1.5461310649084406e-15, -1.6862261357275505e-15, -1.8584981140519205e-16, -4.1207910797100547e-16, 'F', '300', 'P02', '23:00-23:59', 'unknown', '00', 'unknown']
26


**calculate the distance between clusters**

In [35]:
#將群心的數值型資料從str轉為float
for i in range(len(set2_k30_json['cluster_centroids'])):
  for j in range(0,19):
    set2_k30_json['cluster_centroids'][i][j] = float(set2_k30_json['cluster_centroids'][i][j])

In [36]:
## 計算群間總距離

#for numeric data(Euclidean distance) 
#計算資料中心點與所有群心的距離平方和加總
def SquareDsit(x, y):
  return sum([(x-y)**2 for (x,y) in zip(x,y)])

numeric_dist_between_cluters = 0
for i in range(len(set2_k30_json['cluster_centroids'])):
  distance = count_per_cluster[i]*(SquareDsit(set2_k30_json['cluster_centroids'][i][:19], dataset_centroids[:19]))
  numeric_dist_between_cluters += distance



#for categorical data
temp = 0
categorical_dist_between_cluters = 0
for i in range(len(set2_k30_json['cluster_centroids'])):
  for j in range(len(set2_k30_json['cluster_centroids'][i][19:])):
    if (set2_k30_json['cluster_centroids'][i][19+j] == dataset_centroids[19+j]):
      temp += 0
    elif (set2_k30_json['cluster_centroids'][i][19+j] != dataset_centroids[19+j]):
      temp += 1    
  distance = count_per_cluster[i]*temp
  temp = 0
  categorical_dist_between_cluters += distance



#mix numeric and categorical
categorical_weight = set2_k30_json['categorical_weight'] 
dist_between_cluters = numeric_dist_between_cluters + (categorical_weight * categorical_dist_between_cluters)

**calculate evaluative scores**

In [37]:
dist_within_cluster = set2_k30_json['dist_within_cluster'] 
N = len(set2_k30_df)
k = 30

In [38]:
#evaluate score(based on Calinski-Harabasz Score) -> (distance between clusters/distance within cluster)*(N-k/k-1)
evaluate_score = (dist_between_cluters/(dist_within_cluster))*((N-k)/(k-1))
evaluate_score

15136.46500112132

# **set 3**

# **calculate evaluative scores(based on Calinski-Harabasz Score)**

**load data**

In [39]:
set3_k10_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set3_k10.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
set3_k20_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set3_k20.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
set3_k30_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set3_k30.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})

In [40]:
with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set3_k10.json") as f:
  set3_k10_json = json.load(f)

with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set3_k20.json") as f:
  set3_k20_json = json.load(f)

with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set3_k30.json") as f:
  set3_k30_json = json.load(f)

**count per cluster**

In [54]:
count_per_cluster_df = set3_k30_df['cluster_labels'].value_counts().rename_axis('cluster_num').reset_index(name='counts')
count_per_cluster_df = count_per_cluster_df.sort_values(by=['cluster_num'])
count_per_cluster = count_per_cluster_df['counts'].values.tolist()
print(count_per_cluster)
print(len(count_per_cluster))

[11355, 63, 3984, 13040, 3412, 8156, 26753, 248, 3049, 23255, 5433, 5845, 28464, 22, 10087, 6866, 6, 10129, 12265, 5647, 8163, 15098, 40718, 15898, 4858, 3823, 520, 15614, 17349, 8771]
30


**find the centroid of the dataset**

In [55]:
temp = []

categorical_feature = set3_k30_df[['Gender_Code', 'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code']]
numerical_feature = set3_k30_df.drop(columns=['Gender_Code', 'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code', 'cluster_labels'])

temp.append(set3_k30_df[list(numerical_feature.columns)].mean().values.tolist())
temp.append(categorical_feature.mode().values.tolist())
#print(temp)

dataset_centroids = []
for i in range(len(temp[0])):
  dataset_centroids.append(temp[0][i])

for i in range(len(temp[1][0])):
  dataset_centroids.append(temp[1][0][i])

print(dataset_centroids)
print(len(dataset_centroids))

[-1.9670794114423322e-16, -7.527986994143146e-17, 1.718306190164684e-16, -1.758796545861717e-15, -4.850537872818469e-15, 1.2909304749998321e-15, -1.5461310649084406e-15, -1.6862261357275505e-15, 5.970075325407911e-15, 4.134161138642583e-16, -1.8584981140519205e-16, -4.431347999709824e-15, -7.80571707009734e-17, 2.9021366704742948e-15, 2.4216146902540005e-15, 1.39653646365923e-15, -1.9475798550799014e-15, -1.757137093270125e-15, 4.014940773766894e-15, -3.3101647525748113e-15, 1.8986477486748934e-15, 3.1243448799209925e-15, -3.7961294416707153e-16, -1.2410108446933097e-15, -4.1207910797100547e-16, 'F', '300', 'P02', '23:00-23:59', 'unknown', '00', 'unknown']
32


**calculate the distance between clusters**

In [56]:
#將群心的數值型資料從str轉為float
for i in range(len(set3_k30_json['cluster_centroids'])):
  for j in range(0,25):
    set3_k30_json['cluster_centroids'][i][j] = float(set3_k30_json['cluster_centroids'][i][j])

In [57]:
## 計算群間總距離

#for numeric data(Euclidean distance) 
#計算資料中心點與所有群心的距離平方和加總
def SquareDsit(x, y):
  return sum([(x-y)**2 for (x,y) in zip(x,y)])

numeric_dist_between_cluters = 0
for i in range(len(set3_k30_json['cluster_centroids'])):
  distance = count_per_cluster[i]*(SquareDsit(set3_k30_json['cluster_centroids'][i][:25], dataset_centroids[:25]))
  numeric_dist_between_cluters += distance



#for categorical data
temp = 0
categorical_dist_between_cluters = 0
for i in range(len(set3_k30_json['cluster_centroids'])):
  for j in range(len(set3_k30_json['cluster_centroids'][i][25:])):
    if (set3_k30_json['cluster_centroids'][i][25+j] == dataset_centroids[25+j]):
      temp += 0
    elif (set3_k30_json['cluster_centroids'][i][25+j] != dataset_centroids[25+j]):
      temp += 1    
  distance = count_per_cluster[i]*temp
  temp = 0
  categorical_dist_between_cluters += distance



#mix numeric and categorical
categorical_weight = set3_k30_json['categorical_weight'] 
dist_between_cluters = numeric_dist_between_cluters + (categorical_weight * categorical_dist_between_cluters)

**calculate evaluative scores**

In [58]:
dist_within_cluster = set3_k30_json['dist_within_cluster'] 
N = len(set3_k30_df)
k = 30

In [59]:
#evaluate score(based on Calinski-Harabasz Score) -> (distance between clusters/distance within cluster)*(N-k/k-1)
evaluate_score = (dist_between_cluters/(dist_within_cluster))*((N-k)/(k-1))
evaluate_score

18544.013309859467

# **set 4**

# **calculate evaluative scores(based on Calinski-Harabasz Score)**

**load data**

In [61]:
set4_k10_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set4_k10.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
set4_k20_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set4_k20.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
set4_k30_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/results/set4_k30.csv', dtype={'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})

In [62]:
with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set4_k10.json") as f:
  set4_k10_json = json.load(f)

with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set4_k20.json") as f:
  set4_k20_json = json.load(f)

with open("/content/drive/MyDrive/巨量行銷/kprototypes/outputs/set4_k30.json") as f:
  set4_k30_json = json.load(f)

**count per cluster**

In [76]:
count_per_cluster_df = set4_k30_df['cluster_labels'].value_counts().rename_axis('cluster_num').reset_index(name='counts')
count_per_cluster_df = count_per_cluster_df.sort_values(by=['cluster_num'])
count_per_cluster = count_per_cluster_df['counts'].values.tolist()
print(count_per_cluster)
print(len(count_per_cluster))

[28170, 10863, 76, 6824, 12981, 1252, 29837, 14744, 19520, 5, 175, 3877, 1138, 18057, 249, 512, 5505, 22657, 20090, 30214, 8, 2166, 43690, 7315, 22, 1773, 2874, 13473, 3700, 7124]
30


**find the centroid of the dataset**

In [77]:
temp = []

categorical_feature = set4_k30_df[['Gender_Code', 'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code']]
numerical_feature = set4_k30_df.drop(columns=['Gender_Code', 'ZIP_Code', 'AD_TAG', 'INV_time_mode', 'APP_time_mode', 'TXN_time_mode', 'TXN_zip_code', 'cluster_labels'])

temp.append(set4_k30_df[list(numerical_feature.columns)].mean().values.tolist())
temp.append(categorical_feature.mode().values.tolist())
#print(temp)

dataset_centroids = []
for i in range(len(temp[0])):
  dataset_centroids.append(temp[0][i])

for i in range(len(temp[1][0])):
  dataset_centroids.append(temp[1][0][i])

print(dataset_centroids)
print(len(dataset_centroids))

[-1.9670794114423322e-16, -1.7139122529907155e-16, -7.527986994143146e-17, -3.107525355667985e-16, -1.4407941936690026e-15, 1.8491233143792758e-16, 6.613684236771283e-15, 3.2822980256241275e-16, -2.974190478496039e-16, 3.1704655853791885e-15, -3.135276708072015e-15, -4.856565743603306e-16, 2.46474482276692e-15, 9.185743786567105e-16, -7.636462969578343e-15, 1.718306190164684e-16, -1.758796545861717e-15, -2.4365535375120975e-15, -2.2353854356418053e-15, -4.850537872818469e-15, 1.2909304749998321e-15, -1.5461310649084406e-15, -1.6862261357275505e-15, -4.1677671203003364e-16, -1.484425091250849e-15, 5.970075325407911e-15, 4.134161138642583e-16, -1.8584981140519205e-16, 2.8122900676371733e-16, -1.7730043282354087e-15, -4.431347999709824e-15, -7.80571707009734e-17, 2.9021366704742948e-15, 2.4216146902540005e-15, 1.39653646365923e-15, -1.9475798550799014e-15, -1.757137093270125e-15, 4.014940773766894e-15, -3.3101647525748113e-15, 1.8986477486748934e-15, 3.1243448799209925e-15, -3.79612944167

**calculate the distance between clusters**

In [78]:
#將群心的數值型資料從str轉為float
for i in range(len(set4_k30_json['cluster_centroids'])):
  for j in range(0,44):
    set4_k30_json['cluster_centroids'][i][j] = float(set4_k30_json['cluster_centroids'][i][j])

In [79]:
## 計算群間總距離

#for numeric data(Euclidean distance) 
#計算資料中心點與所有群心的距離平方和加總
def SquareDsit(x, y):
  return sum([(x-y)**2 for (x,y) in zip(x,y)])

numeric_dist_between_cluters = 0
for i in range(len(set4_k30_json['cluster_centroids'])):
  distance = count_per_cluster[i]*(SquareDsit(set4_k30_json['cluster_centroids'][i][:44], dataset_centroids[:44]))
  numeric_dist_between_cluters += distance



#for categorical data
temp = 0
categorical_dist_between_cluters = 0
for i in range(len(set4_k30_json['cluster_centroids'])):
  for j in range(len(set4_k30_json['cluster_centroids'][i][44:])):
    if (set4_k30_json['cluster_centroids'][i][44+j] == dataset_centroids[44+j]):
      temp += 0
    elif (set4_k30_json['cluster_centroids'][i][44+j] != dataset_centroids[44+j]):
      temp += 1    
  distance = count_per_cluster[i]*temp
  temp = 0
  categorical_dist_between_cluters += distance



#mix numeric and categorical
categorical_weight = set4_k30_json['categorical_weight'] 
dist_between_cluters = numeric_dist_between_cluters + (categorical_weight * categorical_dist_between_cluters)

**calculate evaluative scores**

In [81]:
dist_within_cluster = set4_k30_json['dist_within_cluster'] 
N = len(set4_k30_df)
k = 30

In [82]:
#evaluate score(based on Calinski-Harabasz Score) -> (distance between clusters/distance within cluster)*(N-k/k-1)
evaluate_score = (dist_between_cluters/(dist_within_cluster))*((N-k)/(k-1))
evaluate_score

10472.875184241215

# **merge data**

In [85]:
HappyGo_df = pd.read_csv('/content/drive/MyDrive/巨量行銷/kprototypes/dataset/HappyGo_NOV_DEC_0509_FV.csv', dtype={'UID':str, 'Gender_Code':str, 'ZIP_Code':str, 'AD_TAG':str, 'INV_time_mode':str, 'APP_time_mode':str, 'TXN_time_mode':str, 'TXN_zip_code':str})
uid = HappyGo_df['UID']
uid

0         7538002
1         3381995
2         5941571
3         3168811
4          661344
           ...   
308886    9294184
308887    4799178
308888    7725327
308889    4129081
308890    9456107
Name: UID, Length: 308891, dtype: object

In [83]:
cluster_labels = set1_k10_df['cluster_labels']
cluster_labels

0         9
1         8
2         5
3         0
4         8
         ..
308886    1
308887    8
308888    7
308889    2
308890    3
Name: cluster_labels, Length: 308891, dtype: int64

In [87]:
pd.concat([uid, cluster_labels], axis=1)

,UID,cluster_labels
0,7538002,9
1,3381995,8
2,5941571,5
3,3168811,0
4,661344,8
...,...,...
308886,9294184,1
308887,4799178,8
308888,7725327,7
308889,4129081,2
